<a href="https://colab.research.google.com/github/hoangndh136/google_colab_notebook/blob/main/kohya_ss_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Train with Kohya's Stable Diffusion Trainers
%cd /content

from google.colab import drive
drive.mount('/content/drive')

!pip install dadaptation==3.1 diffusers[torch]==0.17.1 easygui==0.98.3 einops==0.6.0 fairscale==0.4.13 ftfy==6.1.1 gradio==3.36.1 huggingface-hub==0.14.1
!pip install lion-pytorch==0.0.6 lycoris_lora==1.8.0.dev6 open-clip-torch==2.20.0 prodigyopt==1.0 pytorch-lightning==1.9.0 safetensors==0.3.1 timm==0.6.12
!pip install tk==0.1.0 transformers==4.30.2 voluptuous==0.13.1 wandb==0.15.0 xformers==0.0.20 omegaconf

%cd /content
!git clone -b 0.41.0 https://github.com/TimDettmers/bitsandbytes
%cd /content/bitsandbytes
!CUDA_VERSION=118 make cuda11x
!python setup.py install

%cd /content
!git clone -b v1.0 https://github.com/camenduru/kohya_ss
%cd /content/kohya_ss

!python kohya_gui.py --share --headless

/content
Mounted at /content/drive
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of accelerate to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of accelerate to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Convert Safetensors to Diffusers
from_safetensors_url = '' #@param {type:"string"}
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/v0.17.1/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget {from_safetensors_url} -O /content/model.safetensors
!python3 convert_original_stable_diffusion_to_diffusers.py --half --from_safetensors --checkpoint_path model.safetensors --dump_path /content/model

In [ ]:
#@title Push to HF.co

import os
from huggingface_hub import create_repo, upload_folder

hf_token = 'HF_WRITE_TOKEN' #@param {type:"string"}
repo_id = 'username/reponame' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
model_path = '/content/train/model' #@param {type:"string"}
upload_folder(folder_path=f'{model_path}', path_in_repo='', repo_id=repo_id, commit_message=commit_message, token=hf_token)

In [ ]:
#@title Push to DagsHub.com

!pip -q install dagshub
from dagshub.upload import Repo, create_repo

repo_id = 'reponame' #@param {type:"string"}
org_name = 'orgname' #@param {type:"string"}
commit_message = '\u2764' #@param {type:"string"}
create_repo(f"{repo_id}", org_name=f"{org_name}")
repo = Repo(f"{org_name}", f"{repo_id}")
model_path = '/content/train/model' #@param {type:"string"}
repo.upload("/content/models", remote_path="data", commit_message=f"{commit_message}", force=True)